<a href="https://colab.research.google.com/github/mahmoudshaddad/Master-/blob/main/test_the_fine_tune_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Standard library import for interacting with the operating system.
import os

# Import PyTorch for deep learning models and tensor computations.
import torch

# Load and manipulate datasets conveniently with Hugging Face's dataset library.
from datasets import load_dataset


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-creden

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    # BitsAndBytesConfig to configure the bitsandbytes library for efficient training.
    BitsAndBytesConfig,
    # HfArgumentParser for parsing command-line arguments to Hugging Face model training scripts.
    HfArgumentParser,
    # TrainingArguments to specify and store hyperparameters for model training.
    TrainingArguments,
    # Pipeline for easy inference with models.
    pipeline,
    # Logging for configuring the logging level of the transformers library.
    logging,
)


In [ ]:
# here we define the model that we use to finetuning job which is LLaMa Chat
# model_name = "NousResearch/Llama-2-7b-chat-hf"
fineModel_name = "MahmoudShaddad/DIN-Llama-2-7b-chat-finetunewith2epoch"
dataset_name = "MahmoudShaddad/Basemodelresults"
tokenizer = AutoTokenizer.from_pretrained(fineModel_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
from datasets import load_dataset, DatasetDict



dataset = load_dataset(dataset_name)

In [ ]:
test_ds= dataset['train']

In [ ]:
import json


file_path = '/content/modelResponetestBaseModel.json'

# Open the file for reading
with open(file_path, 'r') as file:
    # Load the JSON content from the file
    data = json.load(file)

print(data)


['<s>[INST] Text: Die DIN 1045-2 spezifiziert Anforderungen, Bemessung und Konstruktion von Stahlbeton- und Spannbetontragwerken im baulichen Bereich, die aus Normal- und Schwerbeton sowie Hochfestem Beton bestehen.\n\nFrage: Welche Art von Tragwerken werden in der DIN 1045-2 spezifiziert? [/INST]  Nach der Textaussage werden in der DIN 1045-2 folgende Arten von Tragwerken spezifiziert:\n\n1. Stahlbeton-Tragwerken\n2. Spannbetontragwerken\n\nBeide Arten von Tragwerken können aus Normal- und Schwerbeton sowie Hochfestem Beton bestehen.', '<s>[INST] **Frage:** Welche Eigenschaften müssen Bewehrungsstäbe gemäß DIN EN 1992-1-1 erfüllen? [/INST]  According to DIN EN 1992-1-1, beams must meet the following requirements:\n\n1. Fatigue resistance: Beams must be able to withstand fatigue loading without failure.\n2. Tensile strength: Beams must have sufficient tensile strength to resist the loads applied to them.\n3. Compressive strength: Beams must have sufficient compressive strength to resis

In [ ]:
import re
def extract_text(entry):
    match = re.search(r'<s>\[INST\](.*?)\[/INST\]', entry, re.DOTALL)
    if match:
        return match.group(1).strip()  # Returns the matched group, stripped of leading/trailing whitespace
    return ""  # Return an empty string if no match is found

# Extract the desired text from each entry in the data list
extracted_texts = [extract_text(entry) for entry in data]


for text in extracted_texts:
    print(text)
    print("-----")

Text: Die DIN 1045-2 spezifiziert Anforderungen, Bemessung und Konstruktion von Stahlbeton- und Spannbetontragwerken im baulichen Bereich, die aus Normal- und Schwerbeton sowie Hochfestem Beton bestehen.

Frage: Welche Art von Tragwerken werden in der DIN 1045-2 spezifiziert?
-----
**Frage:** Welche Eigenschaften müssen Bewehrungsstäbe gemäß DIN EN 1992-1-1 erfüllen?
-----
Text: DIN 1045-2 behandelt die Bemessung, Konstruktion und Ausführung von Massivbauteilen im baulichen und ingenieurmäßigen Hochbau aus Normalbeton nach den zulässigen Spannungen.

Frage: Welche Aspekte behandelt die DIN 1045-2 im Zusammenhang mit Massivbauteilen im Hochbau?
-----
Frage: Welche Anforderungen werden in DIN EN 1992-1-1 an die Festigkeitsklassen von Beton gestellt?
-----
Frage: Welche Anforderungen werden in der DIN 1045-4 an die Betondeckung gestellt?
-----
Frage: Welche Anforderungen werden in DIN EN 1992-1-1 bezüglich der Bemessung und Konstruktion von Stahlbeton- und Spannbetontragwerken festgelegt?

In [ ]:
pipe = pipeline(task="text-generation", model=fineModel_name, tokenizer=tokenizer, max_length=200)


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
processed_data = []

for text in extracted_texts:
    result = pipe(text)  # Process the text through the pipe function
    processed_data.append({'text': text, 'result': result})

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


In [ ]:

import json



# # # Specify the filename
filename = 'modelResponetestfinetunedModel.json'

# # # Open a file in write mode ('w') and use json.dump() to write the list to the file
with open(filename, 'w', encoding='utf-8') as f:
  json.dump(processed_data, f, ensure_ascii=False, indent=4)
